# Module 3 Homework

## Homework: Vector Search

In this homework, we'll experiemnt with vector with and without Elasticsearch

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Getting the embeddings model

First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)    

In [1]:
# Added code block defining `model_name` variable
model_name: str = "multi-qa-distilbert-cos-v1"

In [2]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

c:\Users\Administrator\anaconda3\envs\conda-demo\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\conda-demo\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Create the embedding for this user question:

In [3]:
user_question = "I just discovered the course. Can I still join it?"

What's the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27

### Notes:

:white_check_mark: 0.07

In [4]:
from typing import TYPE_CHECKING, TypeAlias

if TYPE_CHECKING:
    from numpy import ndarray
    from torch import Tensor


Embedding: TypeAlias = "list[Tensor] | ndarray | Tensor"
v: Embedding = embedding_model.encode(user_question)
print("Q1 Answer: The first value of the resulting vector is {:.4f}".format(v[0]))

Q1 Answer: The first value of the resulting vector is 0.0782


## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

In [5]:
import requests

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/documents-with-ids.json"
docs_url = f"{base_url}/{relative_url}?raw=1"
docs_response = requests.get(docs_url)
documents = docs_response.json()

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents

In [6]:
# Added code block to filter documents
documents = [doc for doc in documents if doc["course"] == "machine-learning-zoomcamp"]

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses.

In [7]:
import numpy as np

from tqdm.auto import tqdm


embeddings: list["Embedding"] = []
for document in tqdm(documents):
    qa_text: str = f"{document['question']} {document['text']}"
    embedding: "Embedding" = embedding_model.encode(qa_text)
    embeddings.append(embedding)

X: "ndarray" = np.array(embeddings)
print("Q2 Answer: The shape of X is {}".format(X.shape))

  0%|          | 0/375 [00:00<?, ?it/s]

Q2 Answer: The shape of X is (375, 768)


## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:

In [8]:
scores = X.dot(v)

What's the highest score in the results?

- 65.0 
- 6.5
- 0.65
- 0.065

### Notes:

:white_check_mark: 0.65

In [9]:
print("Q3 Answer: The highest score in the results is {:.2f}".format(scores.max()))

Q3 Answer: The highest score in the results is 0.65


## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

In [10]:
class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

If you don't understand how the `search` function work:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

In [11]:
import pandas as pd

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/ground-truth-data.csv"
ground_truth_url = f"{base_url}/{relative_url}?raw=1"

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == "machine-learning-zoomcamp"]
ground_truth = df_ground_truth.to_dict(orient="records")

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

### Notes:

:white_check_mark: 0.93

In [12]:
from typing import Callable, TypedDict

from tqdm.auto import tqdm


class Document(TypedDict):
    text: str
    section: str
    question: str
    course: str
    id: str


class GroundTruth(TypedDict):
    question: str
    course: str
    document: str


def relevance_total(
    *,
    ground_truth: list[GroundTruth],
    encode: Callable[[str], "Embedding"],
    search_function: Callable[["Embedding"], list[Document]],
) -> list[list[bool]]:
    relevance_total: list[list[bool]] = []

    for question in tqdm(ground_truth):
        doc_id: str = question["document"]

        question_text: str = question["question"]
        vector: "Embedding" = encode(question_text)
        results: list[Document] = search_function(vector)

        relevance_row: list[bool] = [result["id"] == doc_id for result in results]
        relevance_total.append(relevance_row)

    return relevance_total

In [13]:
from functools import partial


search_function = partial(search_engine.search, num_results=5)

q4_relevance_total: list[list[bool]] = relevance_total(
    ground_truth=ground_truth,
    encode=embedding_model.encode,
    search_function=search_function,
    )

q4_hit_rate: float = sum(any(relevance_row) for relevance_row in q4_relevance_total) / len(q4_relevance_total)
print("Q4 Answer: The hit rate of `VectorSearchEngine` with `num_results=5` is {:.4f}".format(q4_hit_rate))

  0%|          | 0/1830 [00:00<?, ?it/s]

Q4 Answer: The hit rate of `VectorSearchEngine` with `num_results=5` is 0.9399


## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [14]:
from typing import TypedDict


class Mapping(TypedDict):
    text: str
    section: str
    question: str
    course: str
    id: str
    qa_text_vector: "Embedding"


mappings: list[Mapping] = []
for document, embedding in zip(documents, embeddings):
    mapping = document.copy()

    mapping["qa_text_vector"] = embedding
    mappings.append(mapping)

In [15]:
from elasticsearch import Elasticsearch


es_client = Elasticsearch('http://localhost:9200') 

print(es_client.info())

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qa_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine",
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

{'name': '70e192dd1c8c', 'cluster_name': 'docker-cluster', 'cluster_uuid': '4u0tuqftRiWledlCvOGyVA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [16]:
from tqdm.auto import tqdm


for document in tqdm(mappings):
    try:
        es_client.index(index=index_name, document=document)
    except Exception as e:
        print(e)

  0%|          | 0/375 [00:00<?, ?it/s]

In [17]:
from typing import Sequence

from elasticsearch import Elasticsearch


def elastic_search_knn(
    *,
    field: str,
    query_vector: "Embedding",
    course: str,
    es_client: Elasticsearch,
    index: str | Sequence[str] | None,
) -> list["Document"]:
    knn = {
        "field": field,
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 10000,
    }
    if course:
        knn["filter"] = {
            "term": {
                "course": course
            }
        }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"],
    }

    es_results = es_client.search(index=index, body=search_query)

    result_docs = []

    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [18]:
vector_search_term: "Embedding" = embedding_model.encode(user_question)

result_docs: list["Document"] = elastic_search_knn(
    field="qa_text_vector",
    query_vector=vector_search_term,
    course="machine-learning-zoomcamp",
    es_client=es_client,
    index=index_name,
)

for i, doc in enumerate(result_docs):
    print(f"Result document {i + 1}")
    for k, v in doc.items():
        print(f"{k}: {v}")
    print()

Result document 1
question: The course has already started. Can I still join it?
course: machine-learning-zoomcamp
section: General course-related questions
text: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.
id: ee58a693

Result document 2
question: I just joined. What should I do next? How can I access course materials?
course: machine-learning-zoomcamp
section: General course-related questions
text: Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.
Click on the links and start watching the videos. Also watch office hour

In [19]:
print("Q5 Answer: The ID of the document with the highest score is {}".format(result_docs[0]["id"]))

Q5 Answer: The ID of the document with the highest score is ee58a693


## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33

### Notes:

:white_check_mark: 0.93

In [20]:
from typing import Sequence

from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm


def relevance_total_es(
    *,
    ground_truth: list["GroundTruth"],
    model: SentenceTransformer,
    field: str,
    course: str,
    es_client: Elasticsearch,
    index: str | Sequence[str] | None,
) -> list[list[bool]]:
    relevance_total: list[list[bool]] = []

    for question in tqdm(ground_truth):
        doc_id: str = question["document"]

        question_text: str = question["question"]
        vector_search_term: "Embedding" = model.encode(question_text)

        result_docs: list[Document] = elastic_search_knn(
            field=field,
            query_vector=vector_search_term,
            course=course,
            es_client=es_client,
            index=index,
        )

        relevance_row: list[bool] = [result["id"] == doc_id for result in result_docs]
        relevance_total.append(relevance_row)

    return relevance_total

In [21]:
q6_relevance_total: list[list[bool]] = relevance_total_es(
    ground_truth=ground_truth,
    model=embedding_model,
    field="qa_text_vector",
    course="machine-learning-zoomcamp",
    es_client=es_client,
    index=index_name,
    )

q6_hit_rate: float = sum(any(relevance_row) for relevance_row in q6_relevance_total) / len(q6_relevance_total)
print("Q6 Answer: The hitrate for our dataset for Elastic is {:.4f}".format(q6_hit_rate))

  0%|          | 0/1830 [00:00<?, ?it/s]

Q6 Answer: The hitrate for our dataset for Elastic is 0.9399
